## M4-Competition Benchmark Loop

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from utils.m4 import *
from models import TimeSeriesTransformer

ModuleNotFoundError: No module named 'transformer'

---
## Load Data

In [2]:
df_info = pd.read_csv('./M4-methods/Dataset/M4-info.csv')
data_dict = {}
forecast_horizon = {
    'Hourly':48,
    'Daily':14,
    'Weekly':13,
    'Monthly':18,
    'Quarterly':8,
    'Yearly':6,
}
frequency = {
    'Hourly':24,
    'Daily':1,
    'Weekly':1,
    'Monthly':12,
    'Quarterly':4,
    'Yearly':1,
}

USED_PERIODS = ['Weekly'] #['Hourly','Daily','Weekly','Monthly','Quarterly','Yearly']

for SP in USED_PERIODS:
    data_dict[SP] = {
        'train':pd.read_csv(f'M4-methods/Dataset/Train/{SP}-train.csv'),
        'test': pd.read_csv(f'M4-methods/Dataset/Test/{SP}-test.csv')
    }
df_info = df_info[df_info['SP'].isin(USED_PERIODS)]

## Benchmark Loop (Single Serie Training)

Train

In [ ]:
for serie_index in tqdm(range(len(df_info))):
    serie_info = df_info.iloc[serie_index]
    serie_id = serie_info.M4id
    serie_sp = serie_info.SP
    fh = forecast_horizon[serie_sp]
    freq = frequency[serie_sp]
    in_size = 3  # number of points used as input for each forecast
    #
    train_df = data_dict[serie_sp]['train']
    # filtra a serie, remove os valores nan e remove o id da serie(presente no 1 valor)
    ts = train_df[train_df.V1 == serie_id].dropna(axis=1).values.reshape(-1)[1:]
    ts = np.asarray(ts, dtype=np.float32)
    #
    #
    #
    # remove seasonality
    seasonality_in = deseasonalize(ts, freq)

    for i in range(0, len(ts)):
        ts[i] = ts[i] * 100 / seasonality_in[i % freq]

    # detrending
    a, b = detrend(ts)

    for i in range(0, len(ts)):
        ts[i] = ts[i] - ((a * i) + b)
    # scaler = MinMaxScaler()
    # scaler.fit(ts[:-fh].reshape(-1, 1))
    # ts = scaler.transform(ts.reshape(-1, 1)).flatten()
    x_train, y_train, x_test, y_test = split_into_train_test(ts, in_size, fh)
    # MLP benchmark - Produce forecasts
    # y_hat_test_MLP = mlp_bench(x_train.copy(), y_train.copy(), x_test.copy(), fh)

    # ts = scaler.inverse_transform(ts.reshape(-1, 1)).flatten()
    # y_hat_test_MLP = scaler.inverse_transform(y_hat_test_MLP.reshape(-1, 1)).flatten()
    # y_test = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()
    # MLP benchmark - Produce forecasts
    y_hat_test_MLP = mlp_bench(x_train, y_train, x_test, fh)
    for i in range(0, 29):
        y_hat_test_MLP = np.vstack((y_hat_test_MLP, mlp_bench(x_train, y_train, x_test, fh)))
    y_hat_test_MLP = np.median(y_hat_test_MLP, axis=0)

    # add trend
    for i in range(0, len(ts)):
        ts[i] = ts[i] + ((a * i) + b)

    for i in range(0, fh):
        y_hat_test_MLP[i] = y_hat_test_MLP[i] + ((a * (len(ts) + i + 1)) + b)

    # add seasonality
    for i in range(0, len(ts)):
        ts[i] = ts[i] * seasonality_in[i % freq] / 100

    for i in range(len(ts), len(ts) + fh):
        y_hat_test_MLP[i - len(ts)] = y_hat_test_MLP[i - len(ts)] * seasonality_in[i % freq] / 100

    # check if negative or extreme
    for i in range(len(y_hat_test_MLP)):
        if y_hat_test_MLP[i] < 0:
            y_hat_test_MLP[i] = 0
            
        if y_hat_test_MLP[i] > (1000 * max(ts)):
            y_hat_test_MLP[i] = max(ts)   

    x_train, y_train, x_test, y_test = split_into_train_test(ts, in_size, fh)

    
    # Create Train examples for MLP and RNN
   
    # train_x, train_y, test_x, test_y = split_into_train_test(train_serie, test_serie, in_size)
    
    #Naive Forecast (Benchmark)
    
    # pred_y =  naive_predict(train_serie, test_serie, fh)
    # ERRORS['naive_1'][serie_sp]['sMAPE'].append(smape(test_serie, pred_y))
    # ERRORS['naive_1'][serie_sp]['MASE'].append(mase(train_serie, test_serie, pred_y, freq))
    
    # MLP (Benchmark)
    #
    ERRORS['mlp'][serie_sp]['sMAPE'].append(smape(y_test, y_hat_test_MLP))
    ERRORS['mlp'][serie_sp]['MASE'].append(mase(ts[:-fh], y_test, y_hat_test_MLP, freq))  
  



Evaluate

In [ ]:
ERRORS = {
    'naive_1':{p:{'sMAPE':[],'MASE':[]} for p in USED_PERIODS},
    'mlp':{p:{'sMAPE':[],'MASE':[]} for p in USED_PERIODS}
    }

for serie_index in tqdm(range(len(df_info))):
    serie_info = df_info.iloc[serie_index]
    serie_id = serie_info.M4id
    serie_sp = serie_info.SP
    fh = forecast_horizon[serie_sp]
    freq = frequency[serie_sp]
    in_size = 3  # number of points used as input for each forecast
    #
    train_df = data_dict[serie_sp]['train']
    test_df = data_dict[serie_sp]['test']
    # filtra a serie, remove os valores nan e remove o id da serie(presente no 1 valor)
    train_serie = train_df[train_df.V1 == serie_id].dropna(axis=1).values.reshape(-1)[1:]
    test_serie = test_df[test_df.V1 == serie_id].dropna(axis=1).values.reshape(-1)[1:]
    test_serie = test_serie[:fh] # forecast only fh steps
    train_serie = np.asarray(train_serie, dtype=np.float32)
    test_serie = np.asarray(test_serie, dtype=np.float32)
    ts = np.concatenate([train_serie, test_serie], dtype=np.float32)
    #
    #
    #
    # remove seasonality
    seasonality_in = deseasonalize(ts, freq)

    for i in range(0, len(ts)):
        ts[i] = ts[i] * 100 / seasonality_in[i % freq]

    # detrending
    a, b = detrend(ts)

    for i in range(0, len(ts)):
        ts[i] = ts[i] - ((a * i) + b)
    # scaler = MinMaxScaler()
    # scaler.fit(ts[:-fh].reshape(-1, 1))
    # ts = scaler.transform(ts.reshape(-1, 1)).flatten()
    x_train, y_train, x_test, y_test = split_into_train_test(ts, in_size, fh)
    # MLP benchmark - Produce forecasts
    # y_hat_test_MLP = mlp_bench(x_train.copy(), y_train.copy(), x_test.copy(), fh)

    # ts = scaler.inverse_transform(ts.reshape(-1, 1)).flatten()
    # y_hat_test_MLP = scaler.inverse_transform(y_hat_test_MLP.reshape(-1, 1)).flatten()
    # y_test = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()
    # MLP benchmark - Produce forecasts
    y_hat_test_MLP = mlp_bench(x_train, y_train, x_test, fh)
    for i in range(0, 29):
        y_hat_test_MLP = np.vstack((y_hat_test_MLP, mlp_bench(x_train, y_train, x_test, fh)))
    y_hat_test_MLP = np.median(y_hat_test_MLP, axis=0)

    # add trend
    for i in range(0, len(ts)):
        ts[i] = ts[i] + ((a * i) + b)

    for i in range(0, fh):
        y_hat_test_MLP[i] = y_hat_test_MLP[i] + ((a * (len(ts) + i + 1)) + b)

    # add seasonality
    for i in range(0, len(ts)):
        ts[i] = ts[i] * seasonality_in[i % freq] / 100

    for i in range(len(ts), len(ts) + fh):
        y_hat_test_MLP[i - len(ts)] = y_hat_test_MLP[i - len(ts)] * seasonality_in[i % freq] / 100

    # check if negative or extreme
    for i in range(len(y_hat_test_MLP)):
        if y_hat_test_MLP[i] < 0:
            y_hat_test_MLP[i] = 0
            
        if y_hat_test_MLP[i] > (1000 * max(ts)):
            y_hat_test_MLP[i] = max(ts)   

    x_train, y_train, x_test, y_test = split_into_train_test(ts, in_size, fh)

    
    # Create Train examples for MLP and RNN
   
    # train_x, train_y, test_x, test_y = split_into_train_test(train_serie, test_serie, in_size)
    
    #Naive Forecast (Benchmark)
    
    # pred_y =  naive_predict(train_serie, test_serie, fh)
    # ERRORS['naive_1'][serie_sp]['sMAPE'].append(smape(test_serie, pred_y))
    # ERRORS['naive_1'][serie_sp]['MASE'].append(mase(train_serie, test_serie, pred_y, freq))
    
    # MLP (Benchmark)
    #
    ERRORS['mlp'][serie_sp]['sMAPE'].append(smape(y_test, y_hat_test_MLP))
    ERRORS['mlp'][serie_sp]['MASE'].append(mase(ts[:-fh], y_test, y_hat_test_MLP, freq))  
  



In [6]:
ERRORS = {
    'naive_1':{p:{'sMAPE':[],'MASE':[]} for p in USED_PERIODS},
    'mlp':{p:{'sMAPE':[],'MASE':[]} for p in USED_PERIODS}
    }

for serie_index in tqdm(range(len(df_info))):
    serie_info = df_info.iloc[serie_index]
    serie_id = serie_info.M4id
    serie_sp = serie_info.SP
    fh = forecast_horizon[serie_sp]
    freq = frequency[serie_sp]
    in_size = 3  # number of points used as input for each forecast
    #
    train_df = data_dict[serie_sp]['train']
    test_df = data_dict[serie_sp]['test']
    # filtra a serie, remove os valores nan e remove o id da serie(presente no 1 valor)
    train_serie = train_df[train_df.V1 == serie_id].dropna(axis=1).values.reshape(-1)[1:]
    test_serie = test_df[test_df.V1 == serie_id].dropna(axis=1).values.reshape(-1)[1:]
    test_serie = test_serie[:fh] # forecast only fh steps
    train_serie = np.asarray(train_serie, dtype=np.float32)
    test_serie = np.asarray(test_serie, dtype=np.float32)
    ts = np.concatenate([train_serie, test_serie], dtype=np.float32)
    #
    #
    #
    # remove seasonality
    seasonality_in = deseasonalize(ts, freq)

    for i in range(0, len(ts)):
        ts[i] = ts[i] * 100 / seasonality_in[i % freq]

    # detrending
    a, b = detrend(ts)

    for i in range(0, len(ts)):
        ts[i] = ts[i] - ((a * i) + b)
    # scaler = MinMaxScaler()
    # scaler.fit(ts[:-fh].reshape(-1, 1))
    # ts = scaler.transform(ts.reshape(-1, 1)).flatten()
    x_train, y_train, x_test, y_test = split_into_train_test(ts, in_size, fh)
    # MLP benchmark - Produce forecasts
    # y_hat_test_MLP = mlp_bench(x_train.copy(), y_train.copy(), x_test.copy(), fh)

    # ts = scaler.inverse_transform(ts.reshape(-1, 1)).flatten()
    # y_hat_test_MLP = scaler.inverse_transform(y_hat_test_MLP.reshape(-1, 1)).flatten()
    # y_test = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()
    # MLP benchmark - Produce forecasts
    y_hat_test_MLP = mlp_bench(x_train, y_train, x_test, fh)
    for i in range(0, 29):
        y_hat_test_MLP = np.vstack((y_hat_test_MLP, mlp_bench(x_train, y_train, x_test, fh)))
    y_hat_test_MLP = np.median(y_hat_test_MLP, axis=0)

    # add trend
    for i in range(0, len(ts)):
        ts[i] = ts[i] + ((a * i) + b)

    for i in range(0, fh):
        y_hat_test_MLP[i] = y_hat_test_MLP[i] + ((a * (len(ts) + i + 1)) + b)

    # add seasonality
    for i in range(0, len(ts)):
        ts[i] = ts[i] * seasonality_in[i % freq] / 100

    for i in range(len(ts), len(ts) + fh):
        y_hat_test_MLP[i - len(ts)] = y_hat_test_MLP[i - len(ts)] * seasonality_in[i % freq] / 100

    # check if negative or extreme
    for i in range(len(y_hat_test_MLP)):
        if y_hat_test_MLP[i] < 0:
            y_hat_test_MLP[i] = 0
            
        if y_hat_test_MLP[i] > (1000 * max(ts)):
            y_hat_test_MLP[i] = max(ts)   

    x_train, y_train, x_test, y_test = split_into_train_test(ts, in_size, fh)

    
    # Create Train examples for MLP and RNN
   
    # train_x, train_y, test_x, test_y = split_into_train_test(train_serie, test_serie, in_size)
    
    #Naive Forecast (Benchmark)
    
    # pred_y =  naive_predict(train_serie, test_serie, fh)
    # ERRORS['naive_1'][serie_sp]['sMAPE'].append(smape(test_serie, pred_y))
    # ERRORS['naive_1'][serie_sp]['MASE'].append(mase(train_serie, test_serie, pred_y, freq))
    
    # MLP (Benchmark)
    #
    ERRORS['mlp'][serie_sp]['sMAPE'].append(smape(y_test, y_hat_test_MLP))
    ERRORS['mlp'][serie_sp]['MASE'].append(mase(ts[:-fh], y_test, y_hat_test_MLP, freq))  
  



  0%|          | 0/359 [00:00<?, ?it/s]

100%|██████████| 359/359 [08:23<00:00,  1.40s/it]


In [10]:
print("---------FINAL RESULTS---------")
for model, err in ERRORS.items():
    print(f'Model : {model}')
    for sp, sp_err in err.items():
        print(f'  {sp}: ')
        print(f'    sMAPE: {np.mean(sp_err["sMAPE"])*100:.3f}')
        print(f'    MASE: {np.mean(sp_err["MASE"]):.3f}')

---------FINAL RESULTS---------
Model : naive_1
  Weekly: 
    sMAPE: nan
    MASE: nan
Model : mlp
  Weekly: 
    sMAPE: 19.556
    MASE: 13.512


## Benchmark Loop (Multiple Serie Training)

In [ ]:
ERRORS = {
    'naive_1':{p:{'sMAPE':[],'MASE':[]} for p in USED_PERIODS},
    'mlp':{p:{'sMAPE':[],'MASE':[]} for p in USED_PERIODS}
    }


In [14]:
# !pip3 install -r requirements.txt

In [ ]:
ERRORS = {
    'naive_1':{p:{'sMAPE':[],'MASE':[]} for p in USED_PERIODS},
    'mlp':{p:{'sMAPE':[],'MASE':[]} for p in USED_PERIODS}
    }

for serie_index in tqdm(range(len(df_info))):
    serie_info = df_info.iloc[serie_index]
    serie_id = serie_info.M4id
    serie_sp = serie_info.SP
    fh = forecast_horizon[serie_sp]
    freq = frequency[serie_sp]
    in_size = 3  # number of points used as input for each forecast
    #
    train_df = data_dict[serie_sp]['train']
    test_df = data_dict[serie_sp]['test']
    # filtra a serie, remove os valores nan e remove o id da serie(presente no 1 valor)
    train_serie = train_df[train_df.V1 == serie_id].dropna(axis=1).values.reshape(-1)[1:]
    test_serie = test_df[test_df.V1 == serie_id].dropna(axis=1).values.reshape(-1)[1:]
    test_serie = test_serie[:fh] # forecast only fh steps
    train_serie = np.asarray(train_serie, dtype=np.float32)
    test_serie = np.asarray(test_serie, dtype=np.float32)
    ts = np.concatenate([train_serie, test_serie], dtype=np.float32)
    #
    #
    #
    # remove seasonality
    seasonality_in = deseasonalize(ts, freq)

    for i in range(0, len(ts)):
        ts[i] = ts[i] * 100 / seasonality_in[i % freq]

    # detrending
    a, b = detrend(ts)

    for i in range(0, len(ts)):
        ts[i] = ts[i] - ((a * i) + b)
    # scaler = MinMaxScaler()
    # scaler.fit(ts[:-fh].reshape(-1, 1))
    # ts = scaler.transform(ts.reshape(-1, 1)).flatten()
    x_train, y_train, x_test, y_test = split_into_train_test(ts, in_size, fh)
    # MLP benchmark - Produce forecasts
    # y_hat_test_MLP = mlp_bench(x_train.copy(), y_train.copy(), x_test.copy(), fh)

    # ts = scaler.inverse_transform(ts.reshape(-1, 1)).flatten()
    # y_hat_test_MLP = scaler.inverse_transform(y_hat_test_MLP.reshape(-1, 1)).flatten()
    # y_test = scaler.inverse_transform(y_test.reshape(-1, 1)).flatten()
    # MLP benchmark - Produce forecasts
    y_hat_test_MLP = mlp_bench(x_train, y_train, x_test, fh)
    for i in range(0, 29):
        y_hat_test_MLP = np.vstack((y_hat_test_MLP, mlp_bench(x_train, y_train, x_test, fh)))
    y_hat_test_MLP = np.median(y_hat_test_MLP, axis=0)

    # add trend
    for i in range(0, len(ts)):
        ts[i] = ts[i] + ((a * i) + b)

    for i in range(0, fh):
        y_hat_test_MLP[i] = y_hat_test_MLP[i] + ((a * (len(ts) + i + 1)) + b)

    # add seasonality
    for i in range(0, len(ts)):
        ts[i] = ts[i] * seasonality_in[i % freq] / 100

    for i in range(len(ts), len(ts) + fh):
        y_hat_test_MLP[i - len(ts)] = y_hat_test_MLP[i - len(ts)] * seasonality_in[i % freq] / 100

    # check if negative or extreme
    for i in range(len(y_hat_test_MLP)):
        if y_hat_test_MLP[i] < 0:
            y_hat_test_MLP[i] = 0
            
        if y_hat_test_MLP[i] > (1000 * max(ts)):
            y_hat_test_MLP[i] = max(ts)   

    x_train, y_train, x_test, y_test = split_into_train_test(ts, in_size, fh)

    
    # Create Train examples for MLP and RNN
   
    # train_x, train_y, test_x, test_y = split_into_train_test(train_serie, test_serie, in_size)
    
    #Naive Forecast (Benchmark)
    
    # pred_y =  naive_predict(train_serie, test_serie, fh)
    # ERRORS['naive_1'][serie_sp]['sMAPE'].append(smape(test_serie, pred_y))
    # ERRORS['naive_1'][serie_sp]['MASE'].append(mase(train_serie, test_serie, pred_y, freq))
    
    # MLP (Benchmark)
    #
    ERRORS['mlp'][serie_sp]['sMAPE'].append(smape(y_test, y_hat_test_MLP))
    ERRORS['mlp'][serie_sp]['MASE'].append(mase(ts[:-fh], y_test, y_hat_test_MLP, freq))  
  

